<a href="https://colab.research.google.com/github/nirb28/llm/blob/main/bac/rag/test-graph-neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://blog.google/products/search/introducing-knowledge-graph-things-not/

https://www.microsoft.com/en-us/research/blog/graphrag-unlocking-llm-discovery-on-narrative-private-data/

https://github.com/langchain-ai/langchain-experimental/blob/main/libs/experimental/langchain_experimental/graph_transformers/llm.py

In [5]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

In [9]:
import sys
from dotenv import load_dotenv
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Colab!")
    from google.colab import userdata
else:
    print("Not running in Colab.")
    load_dotenv()

def get_secret(secret_name):
    if IN_COLAB:
        userdata.get(secret_name)
    else:
        os.getenv(secret_name)

Not running in Colab.


In [1]:
## Graphdb configuration
NEO4J_URI="neo4j+s://0dfd465c.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="HUelZeF8lKUEDCaiT7wo2lHokOARk2fbAYB0S109tSk"

In [2]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

/tmp/ipykernel_843037/3558848663.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [4]:
graph

In [10]:
groq_api_key=get_secret('GROQ_API_KEY')

In [11]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e54b415cac0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e54b415d6c0>, model_name='Gemma2-9b-It', model_kwargs={})

In [12]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [13]:
!pip install --upgrade --quiet langchain_experimental

In [15]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [16]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [17]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Kimbal Musk', type='Person', properties={}), Node(id='Spacex', type='Organization', properties={}), Node(id='Tesla, Inc.', type='Organization', properties={}), Node(id='X Corp.', type='Organization', properties={}), Node(id='Twitter', type='Organization', properties={}), Node(id='The Boring Company', type='Organization', properties={}), Node(id='Xai', type='Organization', properties={}), Node(id='Neuralink', type='Organization', properties={}), Node(id='Openai', type='Organization', properties={}), Node(id='University Of Pretoria', type='Educational_institution', properties={}), Node(id="Queen'S University", type='Educational_institution', properties={}), Node(id='University Of Pennsylvania', type='Educational_institution', properties={}), Node(id='Stanford University', type='Educational_institution'

In [18]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal Musk', type='Person', properties={}),
 Node(id='Spacex', type='Organization', properties={}),
 Node(id='Tesla, Inc.', type='Organization', properties={}),
 Node(id='X Corp.', type='Organization', properties={}),
 Node(id='Twitter', type='Organization', properties={}),
 Node(id='The Boring Company', type='Organization', properties={}),
 Node(id='Xai', type='Organization', properties={}),
 Node(id='Neuralink', type='Organization', properties={}),
 Node(id='Openai', type='Organization', properties={}),
 Node(id='University Of Pretoria', type='Educational_institution', properties={}),
 Node(id="Queen'S University", type='Educational_institution', properties={}),
 Node(id='University Of Pennsylvania', type='Educational_institution', properties={}),
 Node(id='Stanford University', type='Educational_institution', prop

In [19]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Kimbal Musk', type='Person', properties={}), type='SIBLING', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Organization', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Organization', properties={}), type='KEY_ROLE', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Organization', properties={}),

In [20]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [21]:
graph

In [22]:
graph.query(movie_query)

[]

In [23]:
graph.refresh_schema()
print(graph.schema)

Node properties:
PERSON {name: STRING, born: INTEGER}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT, embedding: LIST}
Person {name: STRING, id: STRING, embedding: LIST}
Genre {name: STRING}
Document {fileName: STRING, fileSize: INTEGER, fileType: STRING, status: STRING, fileSource: STRING, createdAt: LOCAL_DATE_TIME, updatedAt: LOCAL_DATE_TIME, processingTime: FLOAT, errorMessage: STRING, nodeCount: INTEGER, relationshipCount: INTEGER, model: STRING, is_cancelled: BOOLEAN, total_chunks: INTEGER, processed_chunk: INTEGER, chunkNodeCount: INTEGER, chunkRelCount: INTEGER, entityNodeCount: INTEGER, entityEntityRelCount: INTEGER, communityNodeCount: INTEGER, communityRelCount: INTEGER}
Chunk {id: STRING, fileName: STRING, embedding: LIST, text: STRING, position: INTEGER, length: INTEGER, content_offset: INTEGER}
URL {id: STRING, embedding: LIST}
Repository {id: STRING, embedding: LIST}
Parameter {id: STRING, embedding: LIST}
Model {id: STRING, embedding: LIST}
Function

In [24]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7e54e279b7c0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7e54b415cac0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7e54b415d6c0>, model_name='Gemma2-9b-I

In [25]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[directed:DIRECTED]->(p:Person) RETURN p.name

Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': "I don't know the answer.  \n"}

In [26]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[:IN_GENRE]->(g:Genre) RETURN g.name 

Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Action'}, {'g.name': 'Thriller'}]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': 'Action, Adventure, Thriller \n'}

In [ ]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {id: 'Casino'})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': "I don't know the answer. \n"}

In [ ]:
response=chain.invoke({"query":"Which movie were released in 2008"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)
WHERE m.released = "2008"
RETURN m.title
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer. \n"}

In [ ]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title 

Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Heat'}, {'m.title': 'Casino'}, {'m.title': 'Twelve Monkeys (a.k.a. 12 Monkeys)'}, {'m.title': 'Seven (a.k.a. Se7en)'}, {'m.title': 'Usual Suspects, The'}, {'m.title': 'Hate (Haine, La)'}, {'m.title': 'Braveheart'}, {'m.title': 'Taxi Driver'}, {'m.title': 'Anne Frank Remembered'}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': "I don't know the answer. \n"}

In [ ]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {{title: 'Casino'}})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]